In [78]:
import os
os.chdir(r'C:\Users\Feliz\Desktop\jupyter\Finite_PINN_OpenAccess')
import torch
import time
import math
import scipy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from timeit import default_timer

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

path = 'Package_Square_Weak_Form.mat'

def derivee(y,x):
  return torch.autograd.grad(y,x,torch.ones_like(y),retain_graph=True,create_graph=True)[0]

def som(s):
  return torch.sum(s,dim=-1,keepdim=True)

cuda


In [79]:
class Offline:
    def __init__(self, path):
        Data = scipy.io.loadmat(path)
        # Euclidean Coordinates
        coor = torch.tensor(Data['nodes'], dtype = torch.float32).to(device)
        self.X = coor[:,0:1].to(device)
        self.Y = coor[:,1:2].to(device)
        # LBO Eigenfunctions and their Derivatives
        self.Phi = torch.tensor(Data['phi'], dtype = torch.float32).to(device)
        self.dPhi_dx = torch.tensor(Data['dphi_dx'], dtype = torch.float32).to(device)
        self.dPhi_dy = torch.tensor(Data['dphi_dy'], dtype = torch.float32).to(device)
        # Collocations points
        self.L_pde = torch.arange(len(coor))

    def forward(self, labels, n, **kwargs):
        attribute_map = {'x': ('X', True),'y': ('Y', True),
                 'phi': ('Phi', False),'dphi_dx': ('dPhi_dx', False),'dphi_dy': ('dPhi_dy', False)}
        output = []
        for key, (attr, default) in attribute_map.items():
            if kwargs.get(key, default):
                output.append(getattr(self, attr)[labels.int(), :n])
        return tuple(output)


In [107]:
class Online:
    def __init__(self, path, off, model):
        # Database from Offline Stage
        self.off = off
        # Model of Solution
        self.model = model
        # Physics Parameters
        E, mu = 1.0, 0.3
        # Constitutives
        Cons = E/(1-mu**2)*torch.tensor([[1,mu,0],[mu,1,0],[0,0,0.5*(1-mu)]]).to(device)
        self.c1,self.c2,self.c3 = Cons[0,0],Cons[0,1],Cons[2,2]
        # Collocation labels
        self.L_nbc = torch.nonzero(off.Y.squeeze() == torch.max(off.Y.squeeze()), as_tuple=False).squeeze() #the top surface of the square
        self.L_pde = torch.arange(len(off.X))

    def Engergy_loss(self):

        # U
        x, y, phi, dphi_dx, dphi_dy = self.off.forward(self.L_pde, n_u, phi=True, dphi_dx=True, dphi_dy=True)
        x.requires_grad = True
        y.requires_grad = True
        phi.requires_grad = True

        ux, uy = self.model(x,y,phi)

        epsilon_xx = derivee(ux,x) + som(derivee(ux,phi) * dphi_dx)
        epsilon_xy = derivee(ux,y) + som(derivee(ux,phi) * dphi_dy)
        epsilon_yx = derivee(uy,x) + som(derivee(uy,phi) * dphi_dx)
        epsilon_yy = derivee(uy,y) + som(derivee(uy,phi) * dphi_dy)

        sigma_xx0 = self.c1*epsilon_xx + self.c2*epsilon_yy
        sigma_xy0 = self.c3*(epsilon_xy + epsilon_yx)
        sigma_yx0 = self.c3*(epsilon_xy + epsilon_yx)
        sigma_yy0 = self.c2*epsilon_xx + self.c1*epsilon_yy

        Uinternal = torch.mean(0.5 * (epsilon_xx*sigma_xx0 + epsilon_xy*sigma_xy0 + \
                         epsilon_yx*sigma_yx0 + epsilon_yy*sigma_yy0))
        # W
        x, y, phi = self.off.forward(self.L_nbc, n_u, phi=True)

        ux, uy = self.model(x,y,phi)

        Wexternal = torch.mean(uy * x) # Case 1
        # Wexternal = torch.mean(uy * (0.5 + 0.1*torch.sin(x*2*torch.pi))) # Case 2

        return Uinternal - Wexternal


In [108]:
class MLP_gelu(nn.Module):
    def __init__(self,insize,outsize,width,layers):
        super().__init__()
        self.input_layer = torch.nn.Linear(insize,width)
        self.hidden_layers = torch.nn.ModuleList([])
        self.output_layer = torch.nn.Linear(width,outsize)
        for i in range(layers):
          self.linear_layer = torch.nn.Linear(width,width)
          self.hidden_layers.append(self.linear_layer)
        self.layers = layers

    def forward(self, x): # F.gelu
        layer_out = F.gelu(self.input_layer(x))
        for i in range(self.layers):
            layer_out = F.gelu(self.hidden_layers[i](layer_out))
        layer_out = self.output_layer(layer_out)
        return layer_out

class Finite_PINN(nn.Module):
    def __init__(self, n_sigma, n_u):
        super(Finite_PINN, self).__init__()

        width = 64
        self.NN_eu1 = MLP_gelu(2+n_u,1,width,4)
        self.NN_eu2 = MLP_gelu(2+n_u,1,width,4)

    def forward(self, x, y, u):

        v1 = self.NN_eu1(torch.cat((x,y,u),dim=-1)) * y
        v2 = self.NN_eu2(torch.cat((x,y,u),dim=-1)) * y

        return v1,v2

In [109]:
n_sigma = 4
n_u = 4

OFF = Offline(path)
model = Finite_PINN(n_sigma = n_sigma, n_u = n_u).to(device, dtype = torch.float32)
ON = Online(path,OFF,model)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 200
csize = 4

for ep in range(num_epochs):
    model.train()
    t1 = default_timer()
    pde_mse = 0

    optimizer.zero_grad()

    pde_loss = ON.Engergy_loss()

    Loss = pde_loss

    Loss.backward()
    optimizer.step()

    t2 = default_timer()

    print(f'Epoch:{ep+1}, Time:{t2-t1:.3g}s, Energy Loss:{pde_loss:.3e}')


In [ ]:
model.eval()
X_plot, Y_plot, Phi_plot = OFF.X, OFF.Y, OFF.Phi[:,:n_u]
vx_plot, vy_plot = model(X_plot, Y_plot, Phi_plot)
vx_plot = vx_plot.squeeze().squeeze().detach().cpu()
vy_plot = vy_plot.squeeze().squeeze().detach().cpu()
plt.figure(figsize=(6, 5))
plt.subplot(2, 2, 1)
plt.scatter(X_plot.squeeze().detach().cpu(), Y_plot.squeeze().detach().cpu(), c=vx_plot, s = 1.5, cmap='jet')
plt.title('Ux')
plt.axis('equal')
plt.axis('off')
plt.colorbar(fraction=0.046, pad=0.04)
plt.subplot(2, 2, 2)
plt.scatter(X_plot.squeeze().detach().cpu(), Y_plot.squeeze().detach().cpu(), c=vy_plot, s = 1.5, cmap='jet')
plt.title('Uy')
plt.axis('equal')
plt.axis('off')
plt.colorbar(fraction=0.046, pad=0.04)
plt.tight_layout()
plt.show()